In [1]:
import sys
from pathlib import Path

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")
    
# Set the environment variables from the file <root_dir>/.env
from mlfs import config
settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Added the following directory to the PYTHONPATH: /home/bjpw/workspace/mlfs-book
HopsworksSettings initialized!


<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [2]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from mlfs.airquality import util
from mlfs import config
import json
import os
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [3]:
project = hopsworks.login(engine="python")
fs = project.get_feature_store() 
secrets = hopsworks.get_secrets_api()

# This line will fail if you have not registered the AQICN_API_KEY as a secret in Hopsworks
AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

2025-11-17 08:25:27,815 INFO: Initializing external client
2025-11-17 08:25:27,817 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-17 08:25:29,155 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1271978


'{"country": "germany", "city": "berlin", "street": "mitte--bruckenstrasse", "aqicn_url": "https://api.waqi.info/feed/@10039", "latitude": "40.6947214", "longitude": "-73.9279986"}'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [4]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)
print(weather_fg)

In [5]:
import great_expectations as ge
cloud_cover_expectation_suite = ge.core.ExpectationSuite(
    expectation_suite_name="aq_expectation_suite"
)

def expect_greater_than_zero(suite, col, min, max):
    suite.add_expectation(
        ge.core.ExpectationConfiguration(
            expectation_type="expect_column_min_to_be_between",
            kwargs={
                "column":col,
                "min_value":-0.1,
                "max_value":1000.0,
                "strict_min":True
            }
        )
    )

expect_greater_than_zero(cloud_cover_expectation_suite, "cloud_cover", -0.1, 100)

In [6]:
# Get or create feature group 
cloud_cover_fg = fs.get_or_create_feature_group(
    name='cloud_cover',
    description='Cloud cover of each day',
    version=1,
    primary_key=['city'],
    event_time="date",
    expectation_suite=cloud_cover_expectation_suite
)
print(cloud_cover_fg)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [7]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQICN_API_KEY)
aq_today_df

,pm25,country,city,street,date,url
0,38.0,germany,berlin,mitte--bruckenstrasse,2025-11-17,https://api.waqi.info/feed/@10039


In [8]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float32       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
dtypes: datetime64[ns](1), float32(1), object(4)
memory usage: 172.0+ bytes


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [9]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 40.75°N -74.0°E
Elevation 21.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2025-11-17,1.45,0.0,15.905319,264.805664,berlin
1,2025-11-18,0.10,0.0,11.525623,268.210114,berlin
2,2025-11-19,3.50,0.4,6.489992,33.690102,berlin
3,2025-11-20,5.90,0.0,6.489992,19.440128,berlin
4,2025-11-21,9.10,0.0,6.849467,176.987274,berlin
5,2025-11-22,7.45,1.7,17.727943,12.907409,berlin
6,2025-11-23,11.65,0.0,6.989936,55.491425,berlin


In [10]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         7 non-null      datetime64[ns]
 1   temperature_2m_mean          7 non-null      float32       
 2   precipitation_sum            7 non-null      float32       
 3   wind_speed_10m_max           7 non-null      float32       
 4   wind_direction_10m_dominant  7 non-null      float32       
 5   city                         7 non-null      object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 352.0+ bytes


In [11]:
## <span style='color:#ff5f27'> Get Cloud Cover Forecast data</span>

In [12]:
from dotenv import load_dotenv
import os

load_dotenv(f"{root_dir}/.env", override=True)
meteosource_api_key = os.getenv('METEOSOURCE_API_KEY')
if not meteosource_api_key:
    raise ValueError("METEOSOURCE_API_KEY environment variable is not set")

In [13]:
import pandas as pd
import requests
import os

url = f"https://www.meteosource.com/api/v1/free/point?lat={latitude}&lon={longitude}&sections=daily&language=en&units=metric&key={meteosource_api_key}"
response = requests.get(url)
response.raise_for_status()
data = response.json()

daily_data = data['daily']['data']

records = []
for day in daily_data:
    record = {
        'date': day['day'],
        'cloud_cover': day['all_day']['cloud_cover']['total'],
        'city': city
    }
    records.append(record)

daily_cloud_cover_df = pd.DataFrame(records)

# Convert date column to datetime, then to date
daily_cloud_cover_df['date'] = pd.to_datetime(daily_cloud_cover_df['date']).dt.date

daily_cloud_cover_df

,date,cloud_cover,city
0,2025-11-17,50,berlin
1,2025-11-18,59,berlin
2,2025-11-19,91,berlin
3,2025-11-20,75,berlin
4,2025-11-21,93,berlin
5,2025-11-22,72,berlin
6,2025-11-23,54,berlin


In [14]:
daily_cloud_cover_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         7 non-null      object
 1   cloud_cover  7 non-null      int64 
 2   city         7 non-null      object
dtypes: int64(1), object(2)
memory usage: 296.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [15]:
# Insert new data
air_quality_fg.insert(aq_today_df)

2025-11-17 08:25:34,133 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1271978/fs/1265731/fg/1718776


Uploading Dataframe: 100.00% |███████████████████████████████████████████| Rows 1/1 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: air_quality_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1271978/jobs/named/air_quality_1_offline_fg_materialization/executions


(Job('air_quality_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 758919
         }
       },
       "result": {
         "observed_value": 37.999996185302734,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-17T07:25:34.000133Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "succ

In [16]:
# Insert new data
weather_fg.insert(daily_df, wait=True)

2025-11-17 08:25:47,173 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1271978/fs/1265731/fg/1727818


Uploading Dataframe: 100.00% |███████████████████████████████████████████| Rows 7/7 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1271978/jobs/named/weather_1_offline_fg_materialization/executions
2025-11-17 08:26:04,281 INFO: Waiting for execution to finish. Current state: INITIALIZING. Final status: UNDEFINED
2025-11-17 08:26:07,460 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-17 08:27:33,210 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-17 08:27:33,358 INFO: Waiting for log aggregation to finish.
2025-11-17 08:27:41,941 INFO: Execution finished successfully.


(Job('weather_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "wind_speed_10m_max",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 764929
         }
       },
       "result": {
         "observed_value": 6.489992141723633,
         "element_count": 7,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-17T07:25:47.000172Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": 

In [17]:
# Insert new data
cloud_cover_fg.insert(daily_cloud_cover_df, wait=True)

2025-11-17 08:27:42,138 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1271978/fs/1265731/fg/1721818


Uploading Dataframe: 100.00% |███████████████████████████████████████████| Rows 7/7 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: cloud_cover_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1271978/jobs/named/cloud_cover_1_offline_fg_materialization/executions
2025-11-17 08:27:58,528 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-17 08:28:01,705 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-17 08:29:46,731 INFO: Waiting for execution to finish. Current state: SUCCEEDING. Final status: UNDEFINED
2025-11-17 08:29:49,920 INFO: Waiting for execution to finish. Current state: FINISHED. Final status: SUCCEEDED
2025-11-17 08:29:50,452 INFO: Waiting for log aggregation to finish.
2025-11-17 08:29:50,453 INFO: Execution finished successfully.


(Job('cloud_cover_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "cloud_cover",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 762030
         }
       },
       "result": {
         "observed_value": 50,
         "element_count": 7,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-17T07:27:42.000138Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_ex

In [18]:
cloud_cover_fg.update_feature_description("date", "Date of measurement of weather")
cloud_cover_fg.update_feature_description("city", "City where cloud cover is measured/forecast for")
cloud_cover_fg.update_feature_description("cloud_cover", "Cloud cover in percent")

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
